In [ ]:
import numpy as np
import xarray as xr
import os,errno
import sys
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import salem
import scipy.stats as st
import cartopy.crs as ccrs

dir_data='/Users/lexihenny/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/'

In [ ]:
yrs=np.arange(1980,2020,1)

ds=xr.open_dataset(dir_data+'merra2_gridareas.nc')
#ds=xr.open_dataset(dir_data+'jra55_gridareas_125.nc')
areas=ds['cell_area']/1e6

nums_array_field=[[],[],[],[],[],[],[],[],[],[],[],[]]
areas_array_field=[[],[],[],[],[],[],[],[],[],[],[],[]]
lats_array_field=[[],[],[],[],[],[],[],[],[],[],[],[]]

lats_array_field_nh=[[],[],[],[],[],[],[],[],[],[],[],[]]
lats_array_field_sh=[[],[],[],[],[],[],[],[],[],[],[],[]]

nums_list_field=[]
areas_list_field=[]
lats_list_field=[]

lats_list_field_nh=[]
lats_list_field_sh=[]

for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    if year>=1980:
        #ds=xr.open_dataset(dir_data+'merra2lo.ar.labels.model.'+str(year)+'.nc')
        ds=xr.open_dataset(dir2+'era5tp.ar.labels.model.'+str(year)+'.nc')
        labeled=ds['ar_labeled']
        
        if i==0:
            lat_array=xr.zeros_like(labeled[0,:,:])
            lon_array=xr.zeros_like(labeled[0,:,:])
            for k in range(labeled.lon.size):
                lat_array[:,k]=labeled.lat.values
            for k in range(labeled.lat.size):
                lon_array[k,:]=labeled.lon.values

            weights=np.cos(np.deg2rad(ds.lat))

        num_list_year=[]
        areas_list_year=[]
        lats_list_year=[]
        lats_list_year_nh=[]
        lats_list_year_sh=[]

        nums_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        areas_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        lats_array_year=[[],[],[],[],[],[],[],[],[],[],[],[]]
        lats_array_year_nh=[[],[],[],[],[],[],[],[],[],[],[],[]]
        lats_array_year_sh=[[],[],[],[],[],[],[],[],[],[],[],[]]

        count=0
        for j in range(labeled.time.size):
            if j%100==0:
                print(j)
            date=pd.to_datetime(labeled.time.values[j])
            month=date.month

            label_sel=labeled[j,:,:]
            unique=np.unique(label_sel.values)
            unique=[x for x in unique if x>0]
            num=len(unique)

            num_day=0
            for k in range(len(unique)):
                sel_feature=label_sel.where(label_sel==unique[k])
                area_feature=areas.where(sel_feature>0)
                
                if 1==0:
                    test=area_feature
                    fig=plt.figure(figsize=(20,10))
                    ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
                    cax=ax.pcolormesh(test.lon,test.lat,test,vmin=0,vmax=test.max().values,cmap=plt.cm.seismic)
                    cbar=plt.colorbar(cax,pad=0,fraction=0.046)
                    ax.coastlines(resolution='10m')
                    plt.show()
                    
                area_feature=area_feature.sum(skipna=True).values.tolist()
                lats_sel=lat_array.where(label_sel==unique[k])
                
                if 1==0:
                    test=lats_sel
                    fig=plt.figure(figsize=(20,10))
                    ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
                    cax=ax.pcolormesh(test.lon,test.lat,test,vmin=test.min().values,vmax=test.max().values,cmap=plt.cm.seismic)
                    cbar=plt.colorbar(cax,pad=0,fraction=0.046)
                    ax.coastlines(resolution='10m')
                    plt.show()
                    
                    
                lats_weighted=lats_sel.weighted(weights)
                lat_mean=lats_weighted.mean(skipna=True).values.tolist()

                if area_feature>0:
                    areas_list_year.append(area_feature)
                    lats_list_year.append(abs(lat_mean))
                    areas_array_year[month-1].append(area_feature)
                    lats_array_year[month-1].append(abs(lat_mean))
                    
                    if lat_mean>0:
                        lats_list_year_nh.append(lat_mean)
                        lats_array_year_nh[month-1].append(lat_mean)
                        
                    if lat_mean<0:
                        lats_list_year_sh.append(lat_mean)
                        lats_array_year_sh[month-1].append(lat_mean)
                        
                    num_day=num_day+1

            num_list_year.append(num_day)
            nums_array_year[month-1].append(num_day)
            #sys.exit()
        mean_num=float(sum(num_list_year))/float(len(num_list_year))
        mean_area=float(sum(areas_list_year))/float(len(areas_list_year))
        mean_lat=float(sum(lats_list_year))/float(len(lats_list_year))
        nums_list_field.append(mean_num)
        areas_list_field.append(mean_area)
        lats_list_field.append(mean_lat)
        
        mean_lat_nh=float(sum(lats_list_year_nh))/float(len(lats_list_year_nh))
        mean_lat_sh=float(sum(lats_list_year_sh))/float(len(lats_list_year_sh))
        lats_list_field_nh.append(mean_lat_nh)
        lats_list_field_sh.append(mean_lat_sh)

        for j in range(len(nums_array_year)):
            nums_array_field[j].append(float(sum(nums_array_year[j]))/float(len(nums_array_year[j])))#mean of all time step AR numbers
            areas_array_field[j].append(float(sum(areas_array_year[j]))/float(len(areas_array_year[j])))#mean of all areas in month
            lats_array_field[j].append(float(sum(lats_array_year[j]))/float(len(lats_array_year[j])))#mean of all avg lats in month
            lats_array_field_nh[j].append(float(sum(lats_array_year_nh[j]))/float(len(lats_array_year_nh[j])))#mean of all avg lats in month
            lats_array_field_sh[j].append(float(sum(lats_array_year_sh[j]))/float(len(lats_array_year_sh[j])))#mean of all avg lats in month

In [ ]:
fig=plt.figure(figsize=(12,12))
ax=plt.subplot(1,1,1)
ax.plot(yrs,nums_list_field)
plt.show()

In [ ]:
dk=xr.Dataset()

dk['nums_array_field']=(('months','years'),np.asarray(nums_array_field))
dk['areas_array_field']=(('months','years'),np.asarray(areas_array_field))
dk['lats_array_field']=(('months','years'),np.asarray(lats_array_field))
dk['lats_array_field_nh']=(('months','years'),np.asarray(lats_array_field_nh))
dk['lats_array_field_sh']=(('months','years'),np.asarray(lats_array_field_sh))

dk['nums_list_field']=(('years'),np.asarray(nums_list_field))
dk['areas_list_field']=(('years'),np.asarray(areas_list_field))
dk['lats_list_field']=(('years'),np.asarray(lats_list_field))
dk['lats_list_field_nh']=(('years'),np.asarray(lats_list_field_nh))
dk['lats_list_field_sh']=(('years'),np.asarray(lats_list_field_sh))

dk.coords['years']=np.arange(1980,2020,1)
dk.coords['months']=np.arange(1,13,1)
try:
    os.remove(dir_data+'ar_chars_original_era5tp.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'ar_chars_original_era5tp.nc',mode='w',format='NETCDF4')